In [1]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, select_shares, private_compare, generate_zero_shares_communication
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor

import unittest
import numpy as np
import torch
import importlib

In [2]:
hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [3]:
L = spdz.Q_BITS

def random_as(tensor, mod=L):
    r = torch.LongTensor(tensor.get_shape()).random_(mod)
    return r


In [10]:
y = torch.LongTensor([10]).share(bob, alice)
# def lsb(y, workers):
#     """
#     computes the least significant bit of a shared input tensor
#     """

# "P2 picks a random integer x ∈ ZL−1"
x = torch.LongTensor(y.get_shape()).random_(L-1)
x = decompose(x) # TODO: make sure this is of length L-1 not L

# "and sends ... shares of x[0] over ZL to P0, P1"
x_0 = decompose(x[...,-1])  # pretty sure decompose is backwards... 
x_0 = x_0.share(bob, alice) # least -> greatest from left -> right

# "and sends shares of x over ZL−1 ... to P0, P1"
x = x.share(bob, alice)

# Next, P0, P1 compute shares of r = y + x
r = y + x

# # Next, P0, P1 ... reconstruct r by exchanging shares
# r = r.get().send(bob, alice) # TODO: reconstruct shares remotely INSECURE

r_bob = r.send(bob)

TypeError: Passed in non-pointer<class 'syft.core.frameworks.torch.tensor._LocalTensor'> to GeneralizedPointerTensor

In [ ]:
r_bob = r.child.send(bob)

# xlsb = xbits[..., 0]
# beta = random_as(y, mod=2)
# share x, xlsb, and xbits here, send to the 2 parties


# r = y + x
# r.get()
# rbits = decompose(r)
# rlsb = rbits[..., 0]
# r = r.share(*workers)  # TODO: remove this line when public-private addition works
# beta_prime = private_compare(xbits, r, beta)

# gamma = xor(beta, beta_prime)
# delta = xor(xlsb, rlsb)
# alpha = xor(gamma, delta)
# u = generate_zero_shares_communication(*workers, *alpha.get_shape())
# result = alpha + u
# #     return result
# result

In [7]:
for i in range(100):
    print(private_compare(i+2,i+1, BETA=0, alice=alice, bob=bob).float()[0])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [481]:
(cmpc == 0).sum(1)


 1
 0
 0
 0
 1
 0
 1
 0
 0
[syft.core.frameworks.torch.tensor.ByteTensor of size 9]

In [298]:
np.binary_repr(32, width=31)

'0000000000000000000000000100000'

In [302]:
decompose(torch.LongTensor([32]))



Columns 0 to 12 
    0     0     0     0     0     1     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 30 
    0     0     0     0     0
[syft.core.frameworks.torch.tensor.LongTensor of size 1x31]

In [12]:


# exp = torch.LongTensor([1, 1, 0, 0])



# dimlen = len(x.size())
# while len(beta.size()) < dimlen:
#     beta = beta.unsqueeze(-1)
# beta = beta.expand_as(x)

# t = (r + 1) % (2 ** Q_BITS)

In [293]:


r_bits = decompose(r)
t_bits = decompose(t)

zeros = beta == 0
ones = beta == 1
others = (r == (2 ** Q_BITS - 1)).unsqueeze(-1).expand_as(ones)
ones = ones & (others - 1).long().abs().byte()

c_zeros = _pc_beta0(x[zeros], r_bits[zeros])
c_ones = _pc_beta1(x[ones], t_bits[ones])
c_other = _pc_else(workers, *x.size())

c = (x * 0).long()
c[zeros] = c_zeros
c[ones] = c_ones
c[others] = c_other.get()



s = random_as(c, mod=p)
permute = torch.randperm(c.size(-1))
d = s * c[..., permute]
(d==0).max(dim=1)[0]

RecursionError: maximum recursion depth exceeded while calling a Python object

In [114]:
s



Columns 0 to 12 
    2     0    13     1     9    20    13     1    26    24    12    25    13
   23    20    28    26     8     9    29     7     4     3    17    30     2
    3    21    23     2    17     7     5    10    12     2     2    21    15
    7    20    11    24    15    28     9    21    18     1    25     6     9

Columns 13 to 25 
   16    21     4     0    11    18     1    26    22    29     3    19    21
    4    15    20     4    15    20     4    25    19    24    25    24     5
    7     7    26    16    10    22     4    17    28    16     9    28     9
   23     1    24    20    19     9    11    13    21    30     8     5     7

Columns 26 to 30 
    1    16     5    30    28
   22     6    15     0    23
   18    26    16    18     6
    5    19     4    12    28
[syft.core.frameworks.torch.tensor.LongTensor of size 4x31]

In [21]:
def decompose(tensor):
    """
    decompose a tensor into its binary representation
    """
    powers = torch.arange(0,Q_BITS)
    for i in range(len(tensor.shape)):
        powers = powers.unsqueeze(0)
    tensor = tensor.unsqueeze(-1)
    moduli = 2 ** powers
    tensor = ((tensor+2**(Q_BITS)) / moduli.type_as(tensor)) % 2
    return tensor

def _pc_beta0(x, r):
    # note x and r are both binary tensors,
    # and dim -1 contains their bits
    # x should be shared, r should be public
    w = xor(x, r)
    z = r  - (x - 1)
    w_sum = torch.zeros(w.shape).type_as(w)
    for i in range(Q_BITS - 2, -1, -1):
        w_sum[..., i] = w[..., (i + 1):].sum(dim=-1, keepdim=True)
    c = z + w_sum
    return c


def _pc_beta1(x, t):
    w = xor(x, t)
    z = (x + 1) - t
    w_sum = torch.zeros(w.shape).type_as(w)
    for i in range(Q_BITS - 2, -1, -1):
        w_sum[..., i] = w[..., (i + 1):].sum(dim=-1, keepdim=True)
    c = z + w_sum
    return c

def _pc_else(workers, *sizes):

    u = generate_zero_shares_communication(*workers, *sizes)

    u_ptrdict = get_ptrdict(u)
    (w0, u0), (w1, u1) = u_ptrdict.items()
    u0 = u0.wrap(True)
    u1 = u1.wrap(True)

    c0 = u0 * 0
    c1 = u1 * 0
    for i in range(Q_BITS - 2, -1, -1):
        if i == 0:
            c0[..., i] = -1 * u0[..., i]
            c1[..., i] = u1[..., i]
        c0[..., i] = u0[..., i] + 1
        c1[..., i] = -1 * u1[..., i]
    ptr_dict = {w0:c0.child, w1:c1.child}
    c_gp = _GeneralizedPointerTensor(ptr_dict, torch_type='syft.LongTensor').wrap(True)
    c = _SPDZTensor(c_gp, torch_type='syft.LongTensor').wrap(True)
    return c

def generate_one_shares_communication(alice, bob, sizes):
    return torch.ones(sizes).long().share(alice, bob)

def xor(x, y):
    return x + y - 2 * x * y

def get_ptrdict(mpct):
    child = mpct
    while not isinstance(child, sy.core.frameworks.torch._GeneralizedPointerTensor) and child is not None:
        child = child.child
    if child is None:
        raise TypeError("Expected child tree to contain a GeneralizedPointerTensor")
    return child.pointer_tensor_dict

x = torch.LongTensor([5])#.share(bob, alice)

r = torch.LongTensor([4])

Q_BITS = spdz.Q_BITS

beta = (torch.rand(1)>0).long()

t = (r + 1) % (2 ** Q_BITS)

x_bits = decompose(x)
r_bits = decompose(r)
t_bits = decompose(t)

c_zeros = _pc_beta0(x_bits, r_bits)
c_ones = _pc_beta1(x_bits, t_bits)
c_other = _pc_else([alice, bob], x_bits.shape)

# TODO: recombine c properly here
c = torch.zeros(*x_bits.shape).long()
zeros = (beta == 0).long()
ones = (beta == 1).long()
others = (r == (2 ** Q_BITS - 1)).long()
ones = ones & (others - 1).abs()

gen sz torch.Size([1, 31])


In [22]:
c = (zeros * c_zeros) + (ones * c_ones)

In [ ]:
c[zeros] = c_zeros
c[ones] = c_ones
c[others] = c_other

In [75]:
c_ones



Columns 0 to 12 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 13 to 25 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 30 
    1     1     1     1     1
[syft.core.frameworks.torch.tensor.LongTensor of size 1x31]

In [17]:
def _pc_else(workers, *sizes):
    u = generate_zero_shares_communication(*workers, *sizes)
    (w0, u0), (w1, u1) = u.child.shares.child.pointer_tensor_dict.items()
    u0 = u0.wrap()
    u1 = u1.wrap()
    
    c0 = u0 * 0
    c1 = u1 * 0
    for i in range(Q_BITS - 2, -1, -1):
        if i == 0:
            c0[..., i] = -u0[..., i]
            c1[..., i] = u1[..., i]

        c0[..., i] = u0[..., i] + 1
        c1[..., i] = -u1[..., i]
    ptr_dict = {w0:c0.child, w1:c1.child}
    c_gp = _GeneralizedPointerTensor(ptr_dict, torch_type='syft.LongTensor').wrap(True)
    c = _SPDZTensor(c_gp, torch_type='syft.LongTensor').wrap(True)
    return c

In [21]:
c_other

[Head of chain]
[syft.core.frameworks.torch.tensor.LongTensor with no dimension]

In [66]:
def private_compare(x, r, beta, workers):
    """
    computes beta XOR (x > r)

    x is private input
    r is public input for comparison
    beta is public random bit tensor

    all of type _GeneralizedPointerTensor
    """
    t = (r + 1) % (2 ** Q_BITS)

    x_bits = decompose(x)
    r_bits = decompose(r)
    t_bits = decompose(t)

    zeros = beta == 0
    ones = beta == 1
    others = r == (2 ** Q_BITS - 1)
    ones = ones & (others - 1).abs()

    c_zeros = _pc_beta0(x_bits, r_bits)
    c_ones = _pc_beta1(x_bits, t_bits)
    c_other = _pc_else()

    # TODO: recombine c properly here
    # torch.zeros()
    c = torch.cat([c_zeros, c_ones, c_other], -1)

    s = random_as(c, mod=p)
    permute = torch.randperm(c.size(-1))
    d = s * c[..., permute]
    d.get()
    return (d == 0).max()